In [1]:
import numpy as np
from dataclasses import dataclass
import math
from typing import Literal, List
import gsw  # TEOS-10 library (for salinity)
from datetime import datetime, timezone

In [2]:
from constants import *
from calibration_coefficients import *
from conversion_functions import *
from hexstrings import *

In [3]:
hexstrings = load_hexstrings_simple("basic_emb.hex")
bytes_list_array = [[hex_string[i:i+2] for i in range(0, len(hex_string), 2)] for hex_string in hexstrings]

In [4]:
def temperature(byte_array, index1, index2, index3, coefs):
    F0 = 1000
    temperature_counts = []
    for byte_list in byte_array:
        count = int(byte_list[index1], 16) * 256 + int(byte_list[index2], 16) + int(byte_list[index3], 16) / 256
        temperature_counts.append(count)
    temperature_counts = np.array(temperature_counts)
    temperature_counts = F0 / temperature_counts
    
    temperatures = convert_temperature(temperature_counts, coefs=coefs)
    return temperatures

In [5]:
def pressure(byte_array, index1, index2, index3, start_idx, end_idx, coefs):
    pressure_counts = []
    for byte_list in byte_array:
        count = int(byte_list[index1], 16) * 256 + int(byte_list[index2], 16) + int(byte_list[index3], 16) / 256
        pressure_counts.append(count)
    pressure_counts = np.array(pressure_counts)

    temperature_counts = []
    for hex_string in hexstrings:
        temperature_value = int(hex_string[start_idx:end_idx],16)
        temperature_counts.append(temperature_value)
    temperature_counts = np.array(temperature_counts)
    
    pressure = pressure_from_frequency(pressure_counts, temperature_counts, coefs=coefs)
    return pressure

In [6]:
def conductivity(byte_array,temperatures, pressure, index1, index2, index3, coefs):
    conductivity_counts = []
    for byte_list in byte_array:
        count = int(byte_list[index1], 16) * 256 + int(byte_list[index2], 16) + int(byte_list[index3], 16) / 256
        conductivity_counts.append(count)
    conductivity_counts = np.array(conductivity_counts)

    conductifitys = convert_conductivity(conductivity_counts, temperatures, pressure, coefs=coefs)
    return conductifitys

In [7]:
def salinity(conductifitys, temperatures, pressure):
    salinitys = gsw.SP_from_C(conductifitys, temperatures, pressure)
    return salinitys

In [8]:
def oxygen(temperatures, pressure, salinitys, start_idx, end_idx, coefs, units = "ml/l"):
    voltages = []
    for hex_string in hexstrings:
        v = hex_to_voltage(hex_string[start_idx:end_idx])
        voltages.append(v)
    voltages = np.array(voltages)

    oxygens = convert_sbe43_oxygen(voltages, temperatures, pressure, salinitys, coefs=coefs)
    
    if units == "umol/kg":
        potential_densitys = potential_density_from_t_s_p(temperatures,salinitys,pressure)
        oxygens = convert_oxygen_to_umol_per_kg(oxygens, potential_densitys) *1000
    
    return oxygens

In [9]:
def par(start_idx, end_idx, coefs):
    voltages = []
    for hex_string in hexstrings:
        v = hex_to_voltage(hex_string[start_idx:end_idx])
        voltages.append(v)
    voltages = np.array(voltages)
    
    par = convert_par(voltages, coefs = coefs)
    return par

In [10]:
# Noch nicht überprüft, bzw noch nicht fertig

def altimeter(start_idx, end_idx, coefs, units = "m"):
    voltages = []
    for hex_string in hexstrings:
        v = hex_to_voltage(hex_string[start_idx:end_idx])
        voltages.append(v)
    voltages = np.array(voltages)
    
    alt_m = convert_alimeter(voltages, coefs=coefs, units=units)
    return alt_m

In [11]:
def eco(start_idx, end_idx, coefs):
    voltages = []
    for hex_string in hexstrings:
        v = hex_to_voltage(hex_string[start_idx:end_idx])
        voltages.append(v)
    voltages = np.array(voltages)
    
    eco = convert_eco(voltages, coefs=coefs)
    return eco

In [12]:
def spar(start_idx, end_idx, coefs):
    voltages = []
    for hex_string in hexstrings:
        N = int(hex_string[start_idx:end_idx],16)
        v = N / 819
        voltages.append(v)
    voltages = np.array(voltages)
    spar = convert_spar(voltages, coefs=coefs)
    return spar

In [13]:
def position(start_idx, end_idx):
    navi_hex = []
    latitudes = []
    longitudes = []
    position = []
    for hex_string in hexstrings:
        navi = hex_string[start_idx:end_idx]
        navi_hex.append(navi)
    navi_hex = np.array(navi_hex)
    
    for hex_str in navi_hex:
            # Bytes extrahieren
            bytes_list = [int(hex_str[i:i+2], 16) for i in range(0, len(hex_str)-2, 2)]
            # Bytes aufsplitten
            b1, b2, b3, b4, b5, b6 = bytes_list[:6]
            last_byte_hex = hex_str[-2:]
            b7 = format(int(last_byte_hex, 16), '08b') 
            # Latitude & Longitude berechnen
            lat = ((b1 * 65536 + b2 * 256 + b3) / 50000) * (-1)**int(b7[6])
            lon = ((b4 * 65536 + b5 * 256 + b6) / 50000) * (-1)**int(b7[7])
    
            if b7[1] == 1:
                pos = "new"
            else: 
                pos = "old"
            position.append(pos)
            latitudes.append(lat)
            longitudes.append(lon)
    position = np.array(position)
    latitudes = np.array(latitudes)
    longitudes = np.array(longitudes)
    return position, latitudes, longitudes

In [14]:
def bit_number(start_idx, end_idx, form = '08b'):
    number = []
    
    for hex_string in hexstrings:
        num = format(int(hex_string[start_idx:end_idx], 16), form)
        number.append(num)  
    number = np.array(number)
    return number

In [15]:
def date_time(start_idx, end_idx):
    swapped = [
        "".join([s[80:88][i:i+2] for i in range(0, 8, 2)][::-1])
        for s in hexstrings
    ]
    timestamps = [int(x, 16) for x in swapped]    
    datetimes = [datetime.fromtimestamp(ts, tz=timezone.utc) for ts in timestamps]
    datetimes_formatted = np.array([d.strftime("%Y-%m-%d %H:%M:%S") for d in datetimes])
    return datetimes_formatted

In [16]:
# name 1 = t090C: Temperature [ITS-90, deg C]
# name 2 = t190C: Temperature, 2 [ITS-90, deg C]

temperatures1 = temperature(bytes_list_array, 0, 1, 2, temp1_coefs)
temperatures2 = temperature(bytes_list_array, 9, 10, 11, temp2_coefs)
print("Temperatures1:", temperatures1)
print("Temperatures2:", temperatures2)

# name 0 = prDM: Pressure, Digiquartz [db]  
pressure = pressure(bytes_list_array, 6, 7, 8, 74, 77, pres_coefs)
print("Druck in psi", pressure, "gewünscht: 1.667")
pressure *= PSI_TO_DBAR # Druck von psia zu db laut seabird

print("Druck in dbar:", pressure, "gewünscht: 1.149")

# name 3 = c0mS/cm: Conductivity [mS/cm]
# name 4 = c1mS/cm: Conductivity, 2 [mS/cm]
conductifitys1 = conductivity(bytes_list_array, temperatures1, pressure, 3, 4, 5, cond1_coefs)
conductifitys2 = conductivity(bytes_list_array, temperatures2, pressure, 12, 13, 14, cond2_coefs)
print("Conductifitys1:", conductifitys1)
print("Conductifitys2:", conductifitys2)

# name 15 = sal00: Salinity, Practical [PSU]
# name 16 = sal11: Salinity, Practical, 2 [PSU]
salinitys1 = salinity(conductifitys1, temperatures1, pressure)
salinitys2 = salinity(conductifitys2, temperatures2, pressure)

print("Salinity1:", salinitys1,"PSU")
print("Salinity2:", salinitys2,"PSU")

# name 5 = sbeox0ML/L: Oxygen, SBE 43 [ml/l]
# name 6 = sbeox1ML/L: Oxygen, SBE 43, 2 [ml/l]
# name 7 = sbox0Mm/Kg: Oxygen, SBE 43 [umol/kg]
# name 8 = sbox1Mm/Kg: Oxygen, SBE 43, 2 [umol/kg]
oxygens1 = oxygen(temperatures1, pressure, salinitys1, 30, 33, oxy1_coefs, "ml/l")
oxygens2 = oxygen(temperatures2, pressure, salinitys2, 33, 36, oxy2_coefs, "ml/l")
oxygens3 = oxygen(temperatures1, pressure, salinitys1, 30, 33, oxy1_coefs, "umol/kg")
oxygens4 = oxygen(temperatures2, pressure, salinitys2, 33, 36, oxy2_coefs, "umol/kg")
print("Oxygens1:", oxygens1,"ml/l")
print("Oxygens2:", oxygens2,"ml/l")
print("Oxygens1:", oxygens3,"umol/kg")
print("Oxygens2:", oxygens4,"umol/kg")

# name 11 = par: PAR/Irradiance, Biospherical/Licor
par = par(36, 39, par_coefs)
print("Par:", par, "µmol photons m−2s−1")

# Altimeter
alt_m = altimeter(39, 42, ali_coefs)
print("Alimeter:",alt_m)

# name 9 = flECO-AFL: Fluorescence, WET Labs ECO-AFL/FL [mg/m^3]
flurometer = eco(42 ,45, fluroeco_coefs)
print("Fluorescence:" , flurometer, "mg/m^3")

# name 10 = turbWETntu0: Turbidity, WET Labs ECO [NTU]
turbidity = eco(45 ,48, turbidity_coefs)
print("Turbidity:", turbidity, "NTU")

# name 12 = spar: SPAR, Biospherical/Licor
spar = spar(57, 60, spar_coefs)
print("SPar:", spar, "µmol photons m−2s−1")

# Fluorescence, WET Labs CDOM
flurometer2 = eco(51 ,54, fluroeco_coefs2)
print("Fluorescence:" , flurometer2, "mg/m^3")

# Position
position, latitudes, longitudes = position(60, 74)
print("Position:", position)  
print("Latitudes:", latitudes)
print("Longitudes:",longitudes)

# name 13 = timeS: Time, Elapsed [seconds]
time_elapsed = np.array(Time_elapsed(hexstrings))
print("Time_Elapsed:",time_elapsed,"s")

# name 17 = flag:  0.000e+00
flag = np.zeros_like(hexstrings, dtype=float)
print(flag)

# CTD status
CTD_status_list =  bit_number(77, 78, '04b')
print("CTD status:", CTD_status_list)

# 8-bit number from CTD
bit_list = bit_number(78, 80, '08b')
print("8-Bit Zahl:", bit_list)


# Date and Time
datetimes_formatted = date_time(80, 88)
print("Date and Times", datetimes_formatted)

Temperatures1: [3.56156397 3.56140342 3.56172452 ... 3.74795321 3.74779314 3.74800656]
Temperatures2: [3.56104427 3.56104427 3.56093531 ... 3.7434417  3.74327874 3.7434417 ]
Druck in psi [ 1.66650044  1.73668555  1.73668555 ... 34.06328618 34.12490155
 34.12490155] gewünscht: 1.667
Druck in dbar: [ 1.14901189  1.19740283  1.19740283 ... 23.4858149  23.52829721
 23.52829721] gewünscht: 1.149
Conductifitys1: [15.3307893  15.33055485 15.33078936 ... 17.6564699  17.66039193
 17.66259815]
Conductifitys2: [15.32989468 15.33007497 15.33002992 ... 17.35461101 17.351426
 17.34594621]
Salinity1: [15.55212221 15.55192778 15.55203214 ... 18.02087632 18.02530687
 18.02763887] PSU
Salinity2: [15.5513864  15.55157157 15.55157415 ... 17.68843558 17.68498031
 17.67881762] PSU
Oxygens1: [8.12350183 8.12840859 8.12832994 ... 7.871811   7.87637172 7.87620114] ml/l
Oxygens2: [8.1298626  8.12990359 8.12992871 ... 7.88650347 7.88676744 7.88705948] ml/l
Oxygens1: [358.36352017 358.5800309  358.57653718 ... 34

In [17]:
descent_rate = np.gradient(alt_m, time_elapsed)
print(descent_rate)

[ 0.58608059  0.29304029  0.29304029 ...  0.         -0.29304029
 -0.58608059]


In [28]:
x = depth_from_pressure(pressure, latitudes)
print(x)
descent_rate1 = np.gradient(x, time_elapsed)
print(descent_rate)

window_size = 48  # 2 Sekunden bei 24 Hz
descent_rate_smooth = np.convolve(descent_rate, np.ones(window_size)/window_size, mode='same')
print(descent_rate_smooth)

dz = np.diff(x)
dtM = np.diff(time_elapsed)

descent_rate2 = dz/dtM
print(descent_rate2)
descent_rate_smooth2 = np.convolve(descent_rate2, np.ones(window_size)/window_size, mode='same')
np.set_printoptions(precision=3, suppress=True)
print(descent_rate_smooth2)


[ 1.139  1.187  1.187 ... 23.275 23.317 23.317]
[ 1.151  0.575 -0.575 ... -0.576  0.505  0.   ]
[0.024 0.024 0.036 ... 0.012 0.012 0.004]
[ 1.151  0.    -1.151 ... -2.162  1.01   0.   ]
[1.734723475976807e-16, 0.02397903272916483, 0.02397903272916483, 0.02397903272916483, 0.05138366263955942, 4.510281037539698e-16, 0.023979032729164983, 0.023979032729164983, 0.02397903272916499, 0.02397903272916499, 3.3306690738754696e-16, 0.023979032729164865, 0.023979032729164865, 2.0816681711721685e-16, 0.02397903272916474, 2.0816681711721685e-16, 0.023979032729164865, 3.3306690738754696e-16, 0.023979032729164865, 2.0816681711721685e-16, 0.02397903272916474, 2.0816681711721685e-16, 0.05138366263955946, 4.85722573273506e-16, 0.023979032729165017, 4.440892098500626e-16, 4.440892098500626e-16, 0.023979032729165017, -0.02740462991039405, 4.440892098500626e-16, -0.023979032729163963, 0.023979032729165295, 7.216449660063518e-16, -0.023979049941086933, 0.051383662639559985, -0.047958048243208484, 3.3306690

In [19]:
# name 14 = dz/dtM: Descent Rate [m/s]